In [ ]:
%pip install tqdm


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import os
from pathlib import Path
from tqdm import tqdm

# --- 設定 ---
ROOT_DIR = "/workspace/wm/datasets"
TASK = "pusht_noise"
TRAIN_VAL = "val" # "train" or "val"

VIDEO_DIR = f"{ROOT_DIR}/{TASK}/{TRAIN_VAL}/obses"
LAST_FRAME_DIR = f"{ROOT_DIR}/{TASK}/{TRAIN_VAL}/last_frames"

def save_last_frames(video_dir, output_dir):
    """
    動画ディレクトリから各動画の最終フレームを抽出し保存する。
    """
    os.makedirs(output_dir, exist_ok=True)
    
    video_extensions = ['.mp4']
    video_files = [f for f in os.listdir(video_dir) if Path(f).suffix.lower() in video_extensions]
    video_files.sort()
    
    print(f"Found {len(video_files)} videos in {video_dir}")
    
    for video_name in tqdm(video_files, desc="Extracting"):
        video_path = os.path.join(video_dir, video_name)
        output_path = os.path.join(output_dir, f"{Path(video_name).stem}_last.jpg")
        
        if os.path.exists(output_path):
            # すでに画像がある場合はスキップ
            continue
            
        cap = cv2.VideoCapture(video_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        if total_frames > 0:
            # 最終フレームへシーク
            cap.set(cv2.CAP_PROP_POS_FRAMES, total_frames - 1)
            success, frame = cap.read()
            if success:
                cv2.imwrite(output_path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
            else:
                print(f"Warning: Could not read the last frame of {video_name}")
        
        cap.release()

if __name__ == "__main__":
    save_last_frames(VIDEO_DIR, LAST_FRAME_DIR)
    print(f"\n✓ Process completed. Frames saved to: {LAST_FRAME_DIR}")